In [3]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

In [3]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

In [4]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

In [5]:
ventanas = []
for x in range(1,8):
    ventanas.append("ventana_{}".format(x))

In [6]:
columns = ["date","device_id","dia"] + ventanas
auctions = auctions[columns]
clicks = clicks[columns]

In [7]:
#Para distinguir despues del merge 
auctions.rename(columns ={"date": "date_auction"},inplace = True)
clicks.rename(columns ={"date": "date_click"},inplace = True)

In [8]:
auctions_click = pd.merge_asof(clicks,auctions, left_on = "date_click", 
                               right_on = "date_auction", by = "device_id",
                               suffixes = ("_click","_auction"))

In [9]:
del clicks

In [10]:
auctions_click = auctions_click[~auctions_click["date_click"].isnull()]
auctions_click = auctions_click[~auctions_click["date_auction"].isnull()]

---

### De auctions- events me quedo con sólo datos que correspondan a la competencia

In [11]:
auctions_click_competencia = auctions_click.loc[auctions_click["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [12]:
# Ventanas de tres dias
Xs = {}
print("Cargando features")
for ventana_nro in range(1, 7):
    try:
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_relaciones.csv".format(ventana_nro), index_col=0)
        print("Ventana {} agregada".format(ventana_nro))
    except FileNotFoundError:
        print("No se econtro la ventana {}, se generará".format(ventana_nro))
        Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
No se econtro la ventana 1, se generará
No se econtro la ventana 2, se generará
No se econtro la ventana 3, se generará
No se econtro la ventana 4, se generará
No se econtro la ventana 5, se generará
No se econtro la ventana 6, se generará


## Set que se usará para predecir

In [13]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_relaciones.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir
No se encontraron los features usados para predecir, se generará el dataframe requerido


In [14]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[6].columns.values)

## Cantidad de clicks luego de la ultima subasta dentro de la ventana

In [15]:
def cantidad_clicks_desde_ultima_subasta(dataframe,nro_ventana):
    
    idx = dataframe.groupby(["device_id"])["date_auction"].transform(max) == dataframe["date_auction"]
    dataframe = dataframe[idx]
    
    feature= dataframe.groupby(["device_id"])["date_click"].agg({"date_click":"count"})\
                    .rename(columns = {"date_click":"cantidad_clicks_desde_ultima_subasta_hasta_fin_ventana"})
    return feature["cantidad_clicks_desde_ultima_subasta_hasta_fin_ventana"]


In [16]:
features["cantidad_clicks_desde_ultima_subasta_hasta_fin_ventana"] = (cantidad_clicks_desde_ultima_subasta, ["cantidad_clicks_desde_ultima_subasta_hasta_fin_ventana"])

---

In [17]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final,nombre_data_merge):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        print("generando feature en v{}".format(ventana_nro))
       
        condicion = dataframe["ventana_{}_auction".format(ventana_nro)] & dataframe["ventana_{}".format(ventana_nro) +"_"+nombre_data_merge]
        
        feature = generador_feature(dataframe.loc[condicion], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")
        destinos[ventana_nro].fillna(0,inplace = True)

### Genero los features

In [18]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(auctions_click, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7,"click")
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(auctions_click_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8, "click")
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature cantidad_clicks_desde_ultima_subasta_hasta_fin_ventana en ventanas
generando feature en v1


/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


generando feature en v2
generando feature en v3
generando feature en v4
generando feature en v5
generando feature en v6
Generando feature cantidad_clicks_desde_ultima_subasta_hasta_fin_ventana en set de predicciones
generando feature en v7


In [19]:
del auctions_click

---

### Installs

In [20]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

In [21]:
installs = installs[columns]

In [22]:
#Para distinguir despues del merge 
installs.rename(columns ={"date": "date_install"},inplace = True)

In [23]:
auctions_install = pd.merge_asof(installs,auctions, left_on = "date_install", 
                               right_on = "date_auction", by = "device_id",
                               suffixes = ("_install","_auction"))

In [24]:
auctions_install = auctions_install[~auctions_install["date_install"].isnull()]
auctions_install = auctions_install[~auctions_install["date_auction"].isnull()]

### De auctions- install me quedo con sólo datos que correspondan a la competencia

In [25]:
auctions_install_competencia = auctions_install.loc[auctions_install["device_id"].isin(targets_competencia.index)]

## Cantidad de installs luego de la ultima subasta dentro de la ventana

In [26]:
def cantidad_installs_desde_ultima_subasta(dataframe,nro_ventana):
    
    idx = dataframe.groupby(["device_id"])["date_auction"].transform(max) == dataframe["date_auction"]
    dataframe = dataframe[idx]
    
    feature = dataframe.groupby(["device_id"])["date_install"].agg({"date_install":"count"})\
                    .rename(columns = {"date_install":"cantidad_installs_desde_ultima_subasta_hasta_fin_ventana"})
    return feature["cantidad_installs_desde_ultima_subasta_hasta_fin_ventana"]


In [27]:
features["cantidad_installs_desde_ultima_subasta_hasta_fin_ventana"] = (cantidad_installs_desde_ultima_subasta, ["cantidad_installs_desde_ultima_subasta_hasta_fin_ventana"])

---

### Genero los features

In [28]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(auctions_install, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7,"install")
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(auctions_install, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8, "install")
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature cantidad_installs_desde_ultima_subasta_hasta_fin_ventana en ventanas
generando feature en v1


/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


generando feature en v2
generando feature en v3
generando feature en v4
generando feature en v5
generando feature en v6
Generando feature cantidad_installs_desde_ultima_subasta_hasta_fin_ventana en set de predicciones
generando feature en v7


## Guardando

In [6]:
for ventana_nro in range(1, 7):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_relaciones.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_relaciones.csv")